# Transformação dos Dados
Este notebook faz transformação de dados para facilitar a análise e o

tratamento dos dados

In [ ]:
from google.colab import drive
import os

# monta o drive no Google Drive para acesso aos dados
drive.mount('/content/drive/')

# Muda para a pasta de trabalho inicial onde estão os dados
os.chdir("/content/drive/My Drive/Colab Notebooks/TCC/")

In [ ]:
# importação de bibliotecas
import pandas as pd
import numpy as np

pd.set_option('precision', 4)
pd.options.display.max_columns = 16

##Dados da Tabela 1685

In [ ]:
# Carrega os dados da Tabela 1685
Tabela1685 = pd.read_csv('Tabela1685.csv')
print(Tabela1685)

In [ ]:
# criação do DataFrame do Modelo
Modelo = Tabela1685.iloc[:,0:3]
# apaga o ano de 2006, que só serão usados para o cálculo do diferencial
# de um ano para outro
Modelo.drop(Modelo.loc[(Modelo.Ano==2006)].index,inplace=True)
print(Modelo)

In [ ]:
# Adiciona os campos da Tabela1685 ao modelo
for Ano in range(2007,2019):
  Modelo.loc[Modelo['Ano']==Ano,'TB0'] = \
    Tabela1685.loc[Tabela1685['Ano']==Ano,'Número de unidades locais'].array -\
    Tabela1685.loc[Tabela1685['Ano']==(Ano-1),'Número de unidades locais'].array
  Modelo.loc[Modelo['Ano']==Ano,'TB1'] = \
    Tabela1685.loc[Tabela1685['Ano']==Ano,'Número de empresas e outras organizações atuantes'].array -\
    Tabela1685.loc[Tabela1685['Ano']==(Ano-1),'Número de empresas e outras organizações atuantes'].array
  Modelo.loc[Modelo['Ano']==Ano,'TB2'] = \
    Tabela1685.loc[Tabela1685['Ano']==Ano,'Pessoal ocupado total'].array -\
    Tabela1685.loc[Tabela1685['Ano']==(Ano-1),'Pessoal ocupado total'].array
  Modelo.loc[Modelo['Ano']==Ano,'TB3'] = \
    Tabela1685.loc[Tabela1685['Ano']==Ano,'Pessoal ocupado assalariado'].array -\
    Tabela1685.loc[Tabela1685['Ano']==(Ano-1),'Pessoal ocupado assalariado'].array
  Modelo.loc[Modelo['Ano']==Ano,'TB4'] = \
    Tabela1685.loc[Tabela1685['Ano']==Ano,'Pessoal assalariado médio'].array -\
    Tabela1685.loc[Tabela1685['Ano']==(Ano-1),'Pessoal assalariado médio'].array
  Modelo.loc[Modelo['Ano']==Ano,'TB5'] = \
    Tabela1685.loc[Tabela1685['Ano']==Ano,'Salários e outras remunerações'].array -\
    Tabela1685.loc[Tabela1685['Ano']==(Ano-1),'Salários e outras remunerações'].array
  Modelo.loc[Modelo['Ano']==Ano,'TB6'] = \
    Tabela1685.loc[Tabela1685['Ano']==Ano,'Salário médio mensal SM'].array -\
    Tabela1685.loc[Tabela1685['Ano']==(Ano-1),'Salário médio mensal SM'].array
  Modelo.loc[Modelo['Ano']==Ano,'TB7'] = \
    Tabela1685.loc[Tabela1685['Ano']==Ano,'Salário médio mensal'].array -\
    Tabela1685.loc[Tabela1685['Ano']==(Ano-1),'Salário médio mensal'].array
print(Modelo)

In [ ]:
# converte algumas colunas para valor inteiro
Modelo[['TB0','TB1','TB2','TB3','TB5']] = Modelo[['TB0','TB1','TB2','TB3','TB5']].convert_dtypes()
print(Modelo)

## Transformação de dados de Grande Grupo de CBO ('GG')

In [ ]:
# Carrega os dados do CAGED CBO
CAGED_MunicipiosSaldoCBO = pd.read_csv('CAGED_MunicipiosSaldoCBO.csv',
  dtype={'Ano':int,'Município':int,'CBO 2002 Ocupação':str,'Salário':int,'Saldo Mov':int})
CAGED_MunicipiosSaldoCBO.rename(columns={'Saldo Mov': 'SaldoMov'},inplace=True)
CAGED_MunicipiosSaldoCBO

In [ ]:
# ordena pelo CBO para identificar dados incorretos
CAGED_MunicipiosSaldoCBO.sort_values(['CBO 2002 Ocupação'])

In [ ]:
CAGED_MunicipiosSaldoCBO.loc[
  CAGED_MunicipiosSaldoCBO['CBO 2002 Ocupação']=='0000-1']

In [ ]:
# apaga o conteúdo dos campos com dados incorretos
CAGED_MunicipiosSaldoCBO.loc[
  CAGED_MunicipiosSaldoCBO['CBO 2002 Ocupação']=='0000-1',
  'CBO 2002 Ocupação'] = ''

In [ ]:
# verifica se há mais algum dados incorreto
CAGED_MunicipiosSaldoCBO.loc[
  CAGED_MunicipiosSaldoCBO['CBO 2002 Ocupação']!=''].sort_values(
      ['CBO 2002 Ocupação'])

In [ ]:
# cria a coluna Grande Grupo do CBO nos dados do CAGED
CAGED_MunicipiosSaldoCBO.loc[
  CAGED_MunicipiosSaldoCBO['CBO 2002 Ocupação']!='','Grande Grupo'] = \
    CAGED_MunicipiosSaldoCBO.loc[
      CAGED_MunicipiosSaldoCBO['CBO 2002 Ocupação']!='',
                               'CBO 2002 Ocupação'].str[0]
CAGED_MunicipiosSaldoCBO

In [ ]:
# cria um DataFrame totalizando os saldo de movimentação por "Ano","Município" 
# e "Grande Grupo"
TotaisGrandeGrupo = \
  CAGED_MunicipiosSaldoCBO.groupby([
    "Ano","Município","Grande Grupo"]).aggregate(np.sum)
TotaisGrandeGrupo  

In [ ]:
# guarda o sequencial de Grande Grupo, que será concatenado posteriormente com 
# o prefixo "GG" e o sufixo "M"
sequencialGG = ['0','1','2','3','4','5','6','7','8','9']

# adiciona as colunas de Grande Grupo do CBO na tabela do modelo para 
# saldo de movimentação
for i in sequencialGG:
  Modelo['GG'+i+'M']=0

# adiciona as colunas de Grande Grupo do CBO na tabela do modelo para 
# salário
for i in sequencialGG:
  Modelo['GG'+i+'S']=0

#Preenche as colunas de Grande Grupo do CBO na tabela do modelo
for (Ano,Município,GrandeGrupo) in TotaisGrandeGrupo.index:
  Modelo.loc[(Modelo['Ano']==Ano)&(Modelo['Município']==Município),
             'GG'+GrandeGrupo+'M'] = TotaisGrandeGrupo.loc[
                                       (Ano,Município,GrandeGrupo)]['SaldoMov']
  Modelo.loc[(Modelo['Ano']==Ano)&(Modelo['Município']==Município),
             'GG'+GrandeGrupo+'S'] = TotaisGrandeGrupo.loc[
                                       (Ano,Município,GrandeGrupo)]['Salário']

print(Modelo)

##Transformação de dados de SubGrupo Principal do CBO ('SP')

In [ ]:
# Carrega os dados do CAGED CBO
CAGED_MunicipiosSaldoCBO = pd.read_csv('CAGED_MunicipiosSaldoCBO.csv',
  dtype={'Ano':int,'Município':int,'CBO 2002 Ocupação':str,'Salário':int,'Saldo Mov':int})
CAGED_MunicipiosSaldoCBO.rename(columns={'Saldo Mov': 'SaldoMov'},inplace=True) # ???? mudar na obtenção dos dados
CAGED_MunicipiosSaldoCBO

In [ ]:
# ordena pelo CBO para identificar dados incorretos
CAGED_MunicipiosSaldoCBO.sort_values(['CBO 2002 Ocupação'])

In [ ]:
CAGED_MunicipiosSaldoCBO.loc[
  CAGED_MunicipiosSaldoCBO['CBO 2002 Ocupação']=='0000-1']

In [ ]:
# apaga o conteúdo dos campos com dados incorretos
CAGED_MunicipiosSaldoCBO.loc[
  CAGED_MunicipiosSaldoCBO['CBO 2002 Ocupação']=='0000-1',
  'CBO 2002 Ocupação'] = ''

In [ ]:
# verifica se há mais algum dados incorreto
CAGED_MunicipiosSaldoCBO.loc[
  CAGED_MunicipiosSaldoCBO['CBO 2002 Ocupação']!=''].sort_values(
      ['CBO 2002 Ocupação'])

In [ ]:
# cria a coluna SubGrupo Principal do CBO nos dados do CAGED
CAGED_MunicipiosSaldoCBO.loc[
  CAGED_MunicipiosSaldoCBO['CBO 2002 Ocupação']!='','SubGrupo Principal'] = \
  CAGED_MunicipiosSaldoCBO.loc[
    CAGED_MunicipiosSaldoCBO['CBO 2002 Ocupação']!='','CBO 2002 Ocupação'].str[0:2]
CAGED_MunicipiosSaldoCBO

In [ ]:
# cria um DataFrame totalizando os saldo de movimentação por "Ano","Município" 
# e "SubGrupo Principal"
TotaisSubGrupoPrincipal = \
  CAGED_MunicipiosSaldoCBO.groupby([
    "Ano","Município","SubGrupo Principal"]).aggregate(np.sum)
TotaisSubGrupoPrincipal

In [ ]:
# guarda o sequencial de SubGrupo Principal do CBO, que será concatenado 
# posteriormente com o prefixo "SP"
sequencialSP = CAGED_MunicipiosSaldoCBO['SubGrupo Principal'].unique()
sequencialSP = sequencialSP[np.logical_not(pd.isnull(sequencialSP))]
sequencialSP.sort()

# adiciona as colunas de SubGrupo Principal na tabela do modelo (prefixo 'SP')

# Saldo de Movimentação
for i in sequencialSP:
  Modelo['SP'+i+'M']=0

# Salário
for i in sequencialSP:
  Modelo['SP'+i+'S']=0
    
#Preenche as colunas de SubGrupo Principal na tabela do modelo (prefixo 'IS')
for (Ano,Município,SubGrupoPrincipal) in TotaisSubGrupoPrincipal.index:
  Modelo.loc[(Modelo['Ano']==Ano)&(Modelo['Município']==Município),
             'SP'+SubGrupoPrincipal+'M'] = TotaisSubGrupoPrincipal.loc[
                                 (Ano,Município,SubGrupoPrincipal)]['SaldoMov']
  Modelo.loc[(Modelo['Ano']==Ano)&(Modelo['Município']==Município),
             'SP'+SubGrupoPrincipal+'S'] = TotaisSubGrupoPrincipal.loc[
                                 (Ano,Município,SubGrupoPrincipal)]['Salário']

print(Modelo)

##Transformação de dados de IBGE Subsetor ('IS')

In [ ]:
# Carrega os dados do CAGED Subsetores Economicos
CAGED_MunicipiosSaldoSubsetorEconomico = pd.read_csv(
  'CAGED_MunicipiosSaldoSubsetorEconomico.csv',
  dtype={'Ano':int,'Município':int,'IBGE Subsetor':str,'Salário':int,
         'Saldo Mov':int})
CAGED_MunicipiosSaldoSubsetorEconomico.rename(
    columns={'Saldo Mov': 'SaldoMov'},inplace=True)
CAGED_MunicipiosSaldoSubsetorEconomico

In [ ]:
# ordena por IBGE Subsetor para verificar dados incorretos
CAGED_MunicipiosSaldoSubsetorEconomico.sort_values('IBGE Subsetor')

In [ ]:
# exclui dados incorretos
CAGED_MunicipiosSaldoSubsetorEconomico.drop([51694],inplace=True)
CAGED_MunicipiosSaldoSubsetorEconomico.sort_values('IBGE Subsetor')

In [ ]:
# cria um DataFrame totalizando os saldos de movimentação e salário 
# por "Ano","Município" e "IBGE Subsetor"
TotaisIBGESubsetor = CAGED_MunicipiosSaldoSubsetorEconomico.groupby(
    ["Ano","Município","IBGE Subsetor"]).aggregate(np.sum)
TotaisIBGESubsetor

In [ ]:
# guarda o sequencial de IBGE Subsetor, que será concatenado posteriormente 
# com o prefixo "IS"
sequencialIS = CAGED_MunicipiosSaldoSubsetorEconomico['IBGE Subsetor'].unique()
for i in range(0,sequencialIS.size):
  sequencialIS[i] = str(sequencialIS[i]).zfill(2)
sequencialIS.sort()

# adiciona as colunas de IBGE Subsetor na tabela do modelo
for i in sequencialIS:
  Modelo['IS'+i+'M']=0

for i in sequencialIS:
  Modelo['IS'+i+'S']=0
    
#Preenche as colunas de IBGE Subsetor na tabela do modelo
for (Ano,Município,IBGESubsetor) in TotaisIBGESubsetor.index:
  Modelo.loc[(Modelo['Ano']==Ano)&(Modelo['Município']==Município),
             'IS'+IBGESubsetor.zfill(2)+'M'] = \
    TotaisIBGESubsetor.loc[(Ano,Município,IBGESubsetor)]['SaldoMov']
  Modelo.loc[(Modelo['Ano']==Ano)&(Modelo['Município']==Município),
             'IS'+IBGESubsetor.zfill(2)+'S'] = \
    TotaisIBGESubsetor.loc[(Ano,Município,IBGESubsetor)]['Salário']

print(Modelo)

## Transformação de dados FaixaEmpr

In [ ]:
# Carrega os dados do CAGED relacionado ao tamanho da empresa
CAGED_MunicipiosSaldoFaixaEmpr = pd.read_csv('CAGED_MunicipiosSaldoFaixaEmpr.csv',
  dtype={'Ano':int,'Município':int,'Faixa Empr Início Jan':str,'Salário':int,
         'Saldo Mov':int})
CAGED_MunicipiosSaldoFaixaEmpr.rename(
  columns={'Saldo Mov': 'SaldoMov','Faixa Empr Início Jan':'FaixaEmpr'},
  inplace=True)
print(CAGED_MunicipiosSaldoFaixaEmpr)

In [ ]:
# ordena por FaixaEmpr para verificar dados incorretos
CAGED_MunicipiosSaldoFaixaEmpr.sort_values('FaixaEmpr')

In [ ]:
# converte FaixaEmpr para apenas um dígito
CAGED_MunicipiosSaldoFaixaEmpr.FaixaEmpr = \
  CAGED_MunicipiosSaldoFaixaEmpr.FaixaEmpr.apply(lambda x: x[1])
CAGED_MunicipiosSaldoFaixaEmpr

In [ ]:
# cria um DataFrame totalizando os saldos de movimentação e salário 
# por "Ano","Município" e "FaixaEmpr"
TotaisFaixaEmpr = CAGED_MunicipiosSaldoFaixaEmpr.groupby(
    ["Ano","Município","FaixaEmpr"]).aggregate(np.sum)
TotaisFaixaEmpr

In [ ]:
# guarda o sequencial de FaixaEmpr, que será concatenado posteriormente com o prefixo "FE" ???
sequencialFE = CAGED_MunicipiosSaldoFaixaEmpr['FaixaEmpr'].unique()

# adiciona as colunas de FaixaEmpr na tabela do modelo
for i in sequencialFE:
  Modelo['FE'+i+'M']=0

for i in sequencialFE:
  Modelo['FE'+i+'S']=0
    
#Preenche as colunas de FaixaEmpr na tabela do modelo
for (Ano,Município,FaixaEmpr) in TotaisFaixaEmpr.index:
  Modelo.loc[(Modelo['Ano']==Ano)&(Modelo['Município']==Município),
             'FE'+FaixaEmpr+'M'] = \
    TotaisFaixaEmpr.loc[(Ano,Município,FaixaEmpr)]['SaldoMov']
  Modelo.loc[(Modelo['Ano']==Ano)&(Modelo['Município']==Município),
             'FE'+FaixaEmpr+'S'] = \
    TotaisFaixaEmpr.loc[(Ano,Município,FaixaEmpr)]['Salário']

print(Modelo)

## Transformação de dados Grau Instrução

In [ ]:
# Carrega os dados do CAGED relacionado ao Grau de instrução
CAGED_MunicipiosSaldoGrauInstrucao =\
  pd.read_csv('CAGED_MunicipiosSaldoGrauInstrucao.csv',
    dtype={'Ano':int,'Município':int,'Grau Instrução':str,'Salário':int,
           'Saldo Mov':int})
CAGED_MunicipiosSaldoGrauInstrucao.rename(
  columns={'Saldo Mov': 'SaldoMov','Grau Instrução':'GrauInstrução'},
  inplace=True)
print(CAGED_MunicipiosSaldoGrauInstrucao)

In [ ]:
# ordena por FaixaEmpr para verificar dados incorretos
CAGED_MunicipiosSaldoGrauInstrucao.sort_values('GrauInstrução')

In [ ]:
# exclui dados incorretos
CAGED_MunicipiosSaldoGrauInstrucao.drop([7971,7341,5424],inplace=True)
CAGED_MunicipiosSaldoGrauInstrucao.sort_values('GrauInstrução')

In [ ]:
# converte Grau de Instrução para apenas um dígito
CAGED_MunicipiosSaldoGrauInstrucao.GrauInstrução = \
  CAGED_MunicipiosSaldoGrauInstrucao.GrauInstrução.apply(lambda x: x[1])
CAGED_MunicipiosSaldoGrauInstrucao

In [ ]:
# cria um DataFrame totalizando os saldos de movimentação e salário 
# por "Ano","Município" e "GrauInstrução"
TotaisGrauInstrução = CAGED_MunicipiosSaldoGrauInstrucao.groupby(
    ["Ano","Município","GrauInstrução"]).aggregate(np.sum)
TotaisGrauInstrução

In [ ]:
# guarda o sequencial de GrauInstrução, que será concatenado posteriormente com o prefixo "FE" ???
sequencialGI = CAGED_MunicipiosSaldoGrauInstrucao['GrauInstrução'].unique()

# adiciona as colunas de GrauInstrução na tabela do modelo
for i in sequencialGI:
  Modelo['GI'+i+'M']=0

for i in sequencialGI:
  Modelo['GI'+i+'S']=0
    
#Preenche as colunas de GrauInstrução na tabela do modelo
for (Ano,Município,GrauInstrução) in TotaisGrauInstrução.index:
  Modelo.loc[(Modelo['Ano']==Ano)&(Modelo['Município']==Município),
             'GI'+GrauInstrução+'M'] = \
    TotaisGrauInstrução.loc[(Ano,Município,GrauInstrução)]['SaldoMov']
  Modelo.loc[(Modelo['Ano']==Ano)&(Modelo['Município']==Município),
             'GI'+GrauInstrução+'S'] = \
    TotaisGrauInstrução.loc[(Ano,Município,GrauInstrução)]['Salário']

print(Modelo)

## Transformação de dados Raça/Cor

In [ ]:
# Carrega os dados do CAGED relacionado ao Grau de instrução
CAGED_MunicipiosSaldoRacaCor = pd.read_csv('CAGED_MunicipiosSaldoRacaCor.csv',
    dtype={'Ano':int,'Município':int,'Raça Cor':str,'Salário':int,
           'Saldo Mov':int})
CAGED_MunicipiosSaldoRacaCor.rename(
  columns={'Saldo Mov': 'SaldoMov','Raça Cor':'RaçaCor'},
  inplace=True)
print(CAGED_MunicipiosSaldoRacaCor)

In [ ]:
CAGED_MunicipiosSaldoRacaCor.RaçaCor.unique()

In [ ]:
# verifica quantos registros estão com dados incorretos
CAGED_MunicipiosSaldoRacaCor.loc[
  (CAGED_MunicipiosSaldoRacaCor.RaçaCor=='{ñ')|
  (CAGED_MunicipiosSaldoRacaCor.RaçaCor=='99')]

Como são 633 registros, ou seja, 4.2% do total, optamos por excluir esses dados, ao invés de atribuir arbitrária mente uma informação que pode estar incorreta

In [ ]:
# exclui dados incorretos
CAGED_MunicipiosSaldoRacaCor.drop(
  CAGED_MunicipiosSaldoRacaCor.loc[
    (CAGED_MunicipiosSaldoRacaCor.RaçaCor=='{ñ')|
    (CAGED_MunicipiosSaldoRacaCor.RaçaCor=='99')].index,
  inplace=True)
CAGED_MunicipiosSaldoRacaCor.sort_values('RaçaCor')

In [ ]:
# converte Raça/Cor para apenas um dígito
CAGED_MunicipiosSaldoRacaCor.RaçaCor = \
  CAGED_MunicipiosSaldoRacaCor.RaçaCor.apply(lambda x: x[1])
CAGED_MunicipiosSaldoRacaCor

In [ ]:
# cria um DataFrame totalizando os saldos de movimentação e salário 
# por "Ano","Município" e "RaçaCor"
TotaisRaçaCor = CAGED_MunicipiosSaldoRacaCor.groupby(
    ["Ano","Município","RaçaCor"]).aggregate(np.sum)
TotaisRaçaCor

In [ ]:
# guarda o sequencial de RaçaCor, que será concatenado posteriormente
# com o prefixo "RC"
sequencialRC = CAGED_MunicipiosSaldoRacaCor['RaçaCor'].unique()

# adiciona as colunas de RaçaCor na tabela do modelo
for i in sequencialRC:
  Modelo['RC'+i+'M']=0

for i in sequencialRC:
  Modelo['RC'+i+'S']=0
    
#Preenche as colunas de RaçaCor na tabela do modelo
for (Ano,Município,RaçaCor) in TotaisRaçaCor.index:
  Modelo.loc[(Modelo['Ano']==Ano)&(Modelo['Município']==Município),
    'RC'+RaçaCor+'M'] = TotaisRaçaCor.loc[(Ano,Município,RaçaCor)]['SaldoMov']
  Modelo.loc[(Modelo['Ano']==Ano)&(Modelo['Município']==Município),
    'RC'+RaçaCor+'S'] = TotaisRaçaCor.loc[(Ano,Município,RaçaCor)]['Salário']

print(Modelo)

## Transformação de dados Gênero

In [ ]:
# Carrega os dados do CAGED relacionado ao Grau de instrução
CAGED_MunicipiosSaldoSexo = pd.read_csv('CAGED_MunicipiosSaldoSexo.csv',
    dtype={'Ano':int,'Município':int,'Sexo':str,'Salário':int,
           'Saldo Mov':int})
CAGED_MunicipiosSaldoSexo.rename(
    columns={'Saldo Mov': 'SaldoMov'},inplace=True)
print(CAGED_MunicipiosSaldoSexo)

In [ ]:
# verifica se há dados incorretos em Sexo
CAGED_MunicipiosSaldoSexo.Sexo.unique()

In [ ]:
# converte Sexo para apenas um dígito
CAGED_MunicipiosSaldoSexo.Sexo = \
  CAGED_MunicipiosSaldoSexo.Sexo.apply(lambda x: x[1])
CAGED_MunicipiosSaldoSexo

In [ ]:
# cria um DataFrame totalizando os saldos de movimentação e salário 
# por "Ano","Município" e "Sexo"
TotaisSexo = CAGED_MunicipiosSaldoSexo.groupby(
    ["Ano","Município","Sexo"]).aggregate(np.sum)
TotaisSexo

In [ ]:
# guarda o sequencial de Sexo, que será concatenado posteriormente
# com o prefixo "SE"
sequencialS = CAGED_MunicipiosSaldoSexo['Sexo'].unique()

# adiciona as colunas de Sexo na tabela do modelo
for i in sequencialS:
  Modelo['SE'+i+'M']=0

for i in sequencialS:
  Modelo['SE'+i+'S']=0
    
#Preenche as colunas de Sexo na tabela do modelo
for (Ano,Município,Sexo) in TotaisSexo.index:
  Modelo.loc[(Modelo['Ano']==Ano)&(Modelo['Município']==Município),
    'SE'+Sexo+'M'] = TotaisSexo.loc[(Ano,Município,Sexo)]['SaldoMov']
  Modelo.loc[(Modelo['Ano']==Ano)&(Modelo['Município']==Município),
    'SE'+Sexo+'S'] = TotaisSexo.loc[(Ano,Município,Sexo)]['Salário']

print(Modelo)

##Dados do PIB

In [ ]:
# Carrega os dados do PIB dos municípios
PIBMunicipios = pd.read_excel('PIBMunicipios.xlsx',usecols=['Ano','Código do Município','PIB'])[['Código do Município','Ano','PIB']]
PIBMunicipios.rename(columns={'Código do Município':'Município'},inplace=True)
PIBMunicipios.sort_values(['Município','Ano'],ignore_index=True,inplace=True)
print(PIBMunicipios)

In [ ]:
# Calcula o crescimento do PIB de um ano para o outro a patir de 2007
for Ano in range(2007,2019):
  PIBMunicipios.loc[PIBMunicipios['Ano']==Ano,'Crescimento'] = \
    PIBMunicipios.loc[PIBMunicipios['Ano']==Ano,'PIB'].array -\
    PIBMunicipios.loc[PIBMunicipios['Ano']==(Ano-1),'PIB'].array
print(PIBMunicipios)

In [ ]:
# remove as linhas referentes ao ano de 2006
PIBMunicipios.drop(PIBMunicipios.loc[PIBMunicipios['Ano']==2006].index,inplace=True)
# ajusta o código de município com o padrão usada no CAGED, sem o último dígito
PIBMunicipios['Município'] = np.int64(PIBMunicipios['Município']/10)
print(PIBMunicipios)

In [ ]:
# adiciona a coluna de PIB e crescimento ao modelo
Modelo = Modelo.merge(PIBMunicipios,on=['Ano', 'Município'])
print(Modelo)

In [ ]:
# Salvamento para uso futuro
Modelo.to_excel('Modelo.xlsx',index=False)